In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w3-p1/train.csv
/kaggle/input/2021-ai-w3-p1/test.csv
/kaggle/input/2021-ai-w3-p1/submit_sample.csv


In [2]:
import torch
import torch.optim as optim

In [3]:
torch.manual_seed(1)

In [4]:
train = pd.read_csv("/kaggle/input/2021-ai-w3-p1/train.csv")
test = pd.read_csv("/kaggle/input/2021-ai-w3-p1/test.csv")
submit = pd.read_csv("/kaggle/input/2021-ai-w3-p1/submit_sample.csv")

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2557 entries, 0 to 2556
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      2557 non-null   int64  
 1   avgTemp   2557 non-null   float64
 2   minTemp   2557 non-null   float64
 3   maxTemp   2557 non-null   float64
 4   rainFall  2557 non-null   float64
 5   avgPrice  2557 non-null   int64  
dtypes: float64(4), int64(2)
memory usage: 120.0 KB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      24 non-null     int64  
 1   avgTemp   24 non-null     float64
 2   minTemp   24 non-null     float64
 3   maxTemp   24 non-null     float64
 4   rainFall  24 non-null     float64
dtypes: float64(4), int64(1)
memory usage: 1.1 KB


In [7]:
train.head(10)

,year,avgTemp,minTemp,maxTemp,rainFall,avgPrice
0,20100101,-4.9,-11.0,0.9,0.0,2123
1,20100102,-3.1,-5.5,5.5,0.8,2123
2,20100103,-2.9,-6.9,1.4,0.0,2123
3,20100104,-1.8,-5.1,2.2,5.9,2020
4,20100105,-5.2,-8.7,-1.8,0.7,2060
5,20100106,-7.3,-11.4,-2.5,0.3,2060
6,20100107,-6.7,-11.2,-1.2,0.0,2140
7,20100108,-5.6,-11.4,1.4,0.0,2140
8,20100109,-3.1,-8.8,1.8,0.1,2140
9,20100110,-1.3,-5.0,2.8,0.0,2140


In [8]:
X_train = train.drop('avgPrice', axis = 1)
y_train = train['avgPrice']

In [9]:
X_train['month'] = X_train['year']/100 % 100
X_train['month'] = X_train['month'].astype(int)
X_train['year'] = X_train['year'] / 10000 % 100
X_train['year'] = X_train['year'].astype(int)

test['month'] = test['year']/100 % 100
test['month'] = test['month'].astype(int)
test['year'] = test['year'] / 10000 % 100
test['year'] = test['year'].astype(int)

In [10]:
X_train

,year,avgTemp,minTemp,maxTemp,rainFall,month
0,10,-4.9,-11.0,0.9,0.0,1
1,10,-3.1,-5.5,5.5,0.8,1
2,10,-2.9,-6.9,1.4,0.0,1
3,10,-1.8,-5.1,2.2,5.9,1
4,10,-5.2,-8.7,-1.8,0.7,1
...,...,...,...,...,...,...
2552,16,1.2,-3.2,4.5,2.1,12
2553,16,-1.4,-6.5,4.8,0.0,12
2554,16,-1.3,-4.5,2.1,0.2,12
2555,16,-1.4,-7.0,4.6,0.0,12


In [11]:
X_train = torch.FloatTensor(X_train.to_numpy())
y_train = torch.FloatTensor(y_train.to_numpy())
test = torch.FloatTensor(test.to_numpy())

In [12]:
# h = XW 
W = torch.zeros((6,1),requires_grad = True)#설명 변수가 5개
b = torch.zeros(1,requires_grad = True)

optimizer = optim.SGD([W, b], lr = 1e-5)

nb_epochs = 10000

for epoch in range(nb_epochs + 1):
    hypothesis = X_train.matmul(W) + b
    cost = (torch.mean( (hypothesis - y_train)**2 ))
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if(epoch % 100) == 0 :
         print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))
    
    

Epoch    0/10000 hypothesis: tensor([0., 0., 0.,  ..., 0., 0., 0.]) Cost: 12013343.000000
Epoch  100/10000 hypothesis: tensor([ 197.6089,  579.1871,  349.1592,  ...,  978.3387, 1062.1345,
        1350.2122]) Cost: 3758696.500000
Epoch  200/10000 hypothesis: tensor([ 493.4328,  863.6765,  614.1231,  ..., 1421.3109, 1544.1345,
        1815.5953]) Cost: 3197790.250000
Epoch  300/10000 hypothesis: tensor([ 763.1489, 1080.9980,  837.0353,  ..., 1750.3824, 1902.4768,
        2126.3357]) Cost: 2873954.000000
Epoch  400/10000 hypothesis: tensor([ 996.6016, 1264.1827, 1027.7395,  ..., 2025.4447, 2202.0771,
        2380.8379]) Cost: 2637680.500000
Epoch  500/10000 hypothesis: tensor([1197.2472, 1421.0869, 1191.3484,  ..., 2259.7773, 2457.3645,
        2596.8479]) Cost: 2464163.250000
Epoch  600/10000 hypothesis: tensor([1369.5747, 1555.8545, 1331.8163,  ..., 2459.9475, 2675.4907,
        2781.1550]) Cost: 2336612.000000
Epoch  700/10000 hypothesis: tensor([1517.6178, 1671.7061, 1452.4730,  ..., 

In [13]:
predict = test.matmul(W) + b

In [14]:
submit['Expected'] = predict.detach().numpy()
submit

,Id,Expected
0,0,3586.797607
1,1,3334.300537
2,2,3345.123535
3,3,3813.392090
4,4,3798.214600
5,5,3812.595459
6,6,3682.527344
7,7,4065.465576
8,8,3891.979248
9,9,4149.496582


In [15]:
submit.to_csv('submit1.csv',mode = 'w', index = False)